In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [3]:
# Load your dataset
data = pd.read_csv('sensor_raw2.csv')

# Assume you have already trained your model and have the 'model' variable
window_size = 14

In [4]:
# Define the features and labels
features_columns = ['AccX', 'AccY', 'AccZ', 'GyroX', 'GyroY', 'GyroZ']
X = data[features_columns]
y = data['Class']

window_size = 14

def calculate_driver_scores(data, model):
    driver_scores = {}

    for driver_id, driver_data in data.groupby('DriverID'):
        X_driver = []
        y_driver = []
        for _, task_data in driver_data.groupby('Class'):
            for i in range(0, len(task_data) - window_size):
                window = task_data.iloc[i:i + window_size]
                feature_vector = [
                    window['AccX'].mean(),
                    window['AccY'].mean(),
                    window['AccZ'].mean(),
                    window['GyroX'].mean(),
                    window['GyroY'].mean(),
                    window['GyroZ'].mean()
                ]
                X_driver.append(feature_vector)
                y_driver.append(window['Class'].values[0])

        X_train, X_test, y_train, y_test = train_test_split(X_driver, y_driver, test_size=0.2, random_state=42)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Calculate the driver's score based on the predicted behaviors
        driver_score = sum(y_pred) / len(y_pred)  # This is a simplified example; adjust based on your scoring criteria

        # Store the driver's score
        driver_scores[driver_id] = driver_score

    return driver_scores

# Train your model
model = RandomForestClassifier()
model.fit(X, y)

# Calculate and print driver scores
driver_scores = calculate_driver_scores(data, model)

for driver_id, score in driver_scores.items():
    print(f"{driver_id} Score: {score}")

Driver-1 Score: 2.3492063492063493
Driver-2 Score: 2.3870967741935485
Driver-3 Score: 2.803030303030303
